In [35]:

%pip install scikit-learn

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [36]:
df = pd.read_csv('car-details.csv')
df.sample(5)

,name,company,model,edition,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
128,Hyundai Santro Xing GLS,Hyundai,Santro,Xing GLS,2009,Second,Petrol,Individual,Manual,80000,42.12,1086.0,62.10,96.1,5.0,135000
1624,Renault Duster 4x4,Renault,Duster,4x4,2014,Second,Diesel,Individual,Manual,132000,46.34,1461.0,108.45,245.0,5.0,560000
5611,Honda Amaze S Diesel BSIV,Honda,Amaze,S Diesel BSIV,2018,First,Diesel,Individual,Manual,43000,64.40,1498.0,98.63,200.0,5.0,700000
6169,Renault KWID 1.0 RXT 02 Anniversary Edition,Renault,KWID,1.0 RXT 02 Anniversary Edition,2017,First,Petrol,Individual,Manual,30000,54.06,999.0,67.00,91.0,5.0,350000
1556,Mahindra TUV 300 T8,Mahindra,TUV,300 T8,2016,First,Diesel,Individual,Manual,120000,43.44,1493.0,100.00,240.0,7.0,625000


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6926 entries, 0 to 6925
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           6926 non-null   object 
 1   company        6926 non-null   object 
 2   model          6926 non-null   object 
 3   edition        6926 non-null   object 
 4   year           6926 non-null   int64  
 5   owner          6926 non-null   object 
 6   fuel           6926 non-null   object 
 7   seller_type    6926 non-null   object 
 8   transmission   6926 non-null   object 
 9   km_driven      6926 non-null   int64  
 10  mileage_mpg    6718 non-null   float64
 11  engine_cc      6718 non-null   float64
 12  max_power_bhp  6717 non-null   float64
 13  torque_nm      6717 non-null   float64
 14  seats          6718 non-null   float64
 15  selling_price  6926 non-null   int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 865.9+ KB


In [38]:
df.isna().sum()

name               0
company            0
model              0
edition            0
year               0
owner              0
fuel               0
seller_type        0
transmission       0
km_driven          0
mileage_mpg      208
engine_cc        208
max_power_bhp    209
torque_nm        209
seats            208
selling_price      0
dtype: int64

In [39]:
df.shape

(6926, 16)

In [40]:
for col in df.select_dtypes(include='O').columns:
    print(f'Column: {col}')
    print(f'Cardinality: {df[col].nunique()}')
    print(df[col].unique())
    print(df[col].value_counts(normalize=True))
    print()

Column: name
Cardinality: 2058
['Maruti Swift Dzire VDI' 'Skoda Rapid 1.5 TDI Ambition'
 'Honda City 2017-2020 EXi' ... 'Tata Nexon 1.5 Revotorq XT'
 'Ford Freestyle Titanium Plus Diesel BSIV'
 'Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV']
name
Maruti Swift Dzire VDI                          0.017037
Maruti Alto 800 LXI                             0.010973
Maruti Alto LXi                                 0.009962
Maruti Swift VDI                                0.008663
Maruti Swift VDI BSIV                           0.008085
                                                  ...   
Ford EcoSport 1.5 TDCi Titanium Plus BE BSIV    0.000144
Ford Figo 1.5 Sports Edition MT                 0.000144
Volkswagen Polo Select 1.2 MPI Highline         0.000144
Honda Jazz V Diesel                             0.000144
Renault Duster RXZ 110PS AMT BSIV               0.000144
Name: proportion, Length: 2058, dtype: float64

Column: company
Cardinality: 32
['Maruti' 'Skoda' 'Honda' 'Hyundai' 'Toyota' '

In [41]:
df = df.drop(columns=['name', 'model', 'edition'])
df.head()

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
0,Maruti,2014,First,Diesel,Individual,Manual,145500,55.00,1248.0,74.00,190.000000,5.0,450000
1,Skoda,2014,Second,Diesel,Individual,Manual,120000,49.70,1498.0,103.52,250.000000,5.0,370000
2,Honda,2006,Third,Petrol,Individual,Manual,140000,41.60,1497.0,78.00,124.544455,5.0,158000
3,Hyundai,2010,First,Diesel,Individual,Manual,127000,54.06,1396.0,90.00,219.668960,5.0,225000
4,Maruti,2007,First,Petrol,Individual,Manual,120000,37.84,1298.0,88.20,112.776475,5.0,130000


In [42]:
df = df.drop_duplicates()
df.head()

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
0,Maruti,2014,First,Diesel,Individual,Manual,145500,55.00,1248.0,74.00,190.000000,5.0,450000
1,Skoda,2014,Second,Diesel,Individual,Manual,120000,49.70,1498.0,103.52,250.000000,5.0,370000
2,Honda,2006,Third,Petrol,Individual,Manual,140000,41.60,1497.0,78.00,124.544455,5.0,158000
3,Hyundai,2010,First,Diesel,Individual,Manual,127000,54.06,1396.0,90.00,219.668960,5.0,225000
4,Maruti,2007,First,Petrol,Individual,Manual,120000,37.84,1298.0,88.20,112.776475,5.0,130000


In [43]:
df.duplicated().sum()

np.int64(0)

In [44]:
X = df.drop(columns='selling_price')
y = df.selling_price.copy()

print(X.shape, y.shape)

(6907, 12) (6907,)


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5525, 12) (5525,)
(1382, 12) (1382,)


In [46]:
num_cols = X_train.select_dtypes(include='number').columns.tolist()
cat_cols = [col for col in X_train.columns if col not in num_cols]

print(num_cols)
print(cat_cols)

['year', 'km_driven', 'mileage_mpg', 'engine_cc', 'max_power_bhp', 'torque_nm', 'seats']
['company', 'owner', 'fuel', 'seller_type', 'transmission']


In [47]:
num_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

regressor = RandomForestRegressor(
    n_estimators=10, max_depth=5, random_state=42
)

rf_model = Pipeline(steps=[
    ('pre', preprocessor),
    ('reg', regressor)
])
rf_model.fit(X_train, y_train)

,steps,"[('pre', ...), ('reg', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [51]:
from sklearn.metrics import root_mean_squared_error

# Train RMSE
y_train_pred = rf_model.predict(X_train)
train_rmse = root_mean_squared_error(y_train, y_train_pred)
print(f'Train RMSE: {train_rmse:,.3f}')

# Test RMSE
y_test_pred = rf_model.predict(X_test)
test_rmse = root_mean_squared_error(y_test, y_test_pred)
print(f'Test RMSE: {test_rmse:,.3f}')



Train RMSE: 169,947.490
Test RMSE: 172,392.131
